# Исследование надежности заемщиков.

**Описание проекта**

При принятии решения о выдаче кредита одной из наиболее важных характеристик для банка является способность заёмщика вернуть кредит. Необходимо исследовать влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

**Проверим следующие гипотезы:**
 - Если клиент состоит в браке, то вероятность возврата кредита в срок увеличивается.
 - Если в семье есть ребенок, вероятность возврата кредита в срок увеличивается, при этом количество детей не влияет на этот показатель.
 - Чем выше уровень дохода, тем меньше срок возврата кредита.

**Ход исследования:**
 - Обзор данных
 - Предобработка данных
 - Проверка гипотез
 

# Шаг 1. Обзор данных

Составим первое представление о данных платёжеспособности клиентов:

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 11 столбцов и 3 типа данных: float, int, object

Согласно документации:
 - `children` — количество детей в семье
 - `days_employed` — общий трудовой стаж в днях
 - `dob_years` — возраст клиента в годах
 - `education` — уровень образования клиента
 - `education_id` — идентификатор уровня образования
 - `family_status` — семейное положение
 - `family_status_id` — идентификатор семейного положения
 - `gender` — пол клиента
 - `income_type` — тип занятости
 - `debt` — имел ли задолженность по возврату кредитов
 - `total_income` — ежемесячный доход
 - `purpose` — цель получения кредита
 
Стиль названия колонок корректный. В кололонках days_employed и total_income есть пропуски данных.
В столбце days_employed есть положительные и отрицательные значения, необходимо исправить данные на положительное значения. Данные в колонке education имеют разный регистр - необходимо привести данные к единому стилю. 
 

**Выводы**

В таблице представлена информация о клиентах банка, на первый взгляд данной информации будет достаточно для проверки гипотез, но сначала необходимо устранить проблемы в данных. 

# Шаг 2.1 Заполнение пропусков

В столбцах `days_employed` и `total_income` пропущены количественные данные. Из возможных причин пропусков можно выделить:
 - человеческий фактор
 - технические неполадки
 - некорректное объединение

В обоих столбцах незаполненными остались 2174 строки, что составляет 10,09% - это достаточно значительно, поэтому просто удалить данные сроки нельзя, так как это может привести к некорректным выводам. В данном случае лучший вариант - заполнить пропуски характерными значениями, для этого подойдёт среднее арифметическое или медиана.

Для начала заполним пропуски в столбце `total_income`, так как в нём данные не содержат аномалий. Предполагаем, что ежемесячный доход будет зависеть от типа занятости, поэтому пропуски будем заполнять медианным значением по каждой категории из столбца `income_type`.

Узнаем какие категории есть в столбце `income_type`

In [3]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Всего в столбце 8 категорий, количество клиентов в них распределено неравномерно. 
Посчитаем медианный ежемесячный доход для каждой категории:

In [4]:
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Мы видим, что медианный доход сильно отличается, так есть группы с низким доходом - "студенты" и "в декрете" и высоким доходом - "предприниматель". 
Узнаем в каких категориях есть пропуски и заполним их медианным доходом по группе.

In [5]:
df.groupby('income_type')['total_income'].apply(lambda x: x.isna().sum())

income_type
безработный           0
в декрете             0
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
студент               0
Name: total_income, dtype: int64

In [6]:
# Заполняем пропуски медианным доходом по каждой группе
income_type_median = df.groupby(['income_type'])
df['total_income'] = income_type_median.total_income.apply(lambda x: x.fillna(x.median()))
# Проверяем, что все пустые значения заполнены
df.groupby('income_type')['total_income'].apply(lambda x: x.isna().sum())

income_type
безработный        0
в декрете          0
госслужащий        0
компаньон          0
пенсионер          0
предприниматель    0
сотрудник          0
студент            0
Name: total_income, dtype: int64

In [7]:
# Проверяем, что все значения заполнены верно - медиана по каждой группе не должна измениться
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

# Шаг 2.2 Проверка данных на аномалии и исправления.

Получим общую информацию по столбцу `days_employed` с помощью метода describe()

In [8]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

75% данных имеют отрицательное значение, но согласно документации в этом столбце содержатся данные об общем трудовом стаже в днях, что по своей сути не может быть отрицательным числом. Заменим отрицательные значения на положительные.

In [9]:
df['days_employed'] = df['days_employed'].abs()
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Найдем медиану по каждой группе и заполним пропуски 

In [10]:
df.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [11]:
# Заполняем пропуски медианным значением по каждой группе
days_employed_median = df.groupby(['income_type'])
df['days_employed'] = days_employed_median.days_employed.apply(lambda x: x.fillna(x.median()))
# Проверяем, что все пустые значения заполнены
df.groupby('income_type')['days_employed'].apply(lambda x: x.isna().sum())

income_type
безработный        0
в декрете          0
госслужащий        0
компаньон          0
пенсионер          0
предприниматель    0
сотрудник          0
студент            0
Name: days_employed, dtype: int64

Также важно отметить, что в столбце `days_employed` есть аномальные значения: максимальное значение равно 401755 дней (1100 лет), что является невозможным. Так как данные по количеству дней не будут использоваться в исследовании, оставим значения как есть.

Проверим также на аномалии столбец `children`

In [12]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце с данными по количеству детей встречаются аномальные значения: -1 и 20, но так как таких строк менее 1%, мы может их просто удалить.
</div>

In [13]:
df = df.loc[(df['children'] != -1) & (df['children'] != 20)] 
df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Проверим на аномалии также столбцы `dob_years` и `gender`

In [14]:
df['dob_years'].value_counts()

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
53    457
57    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
24    263
64    263
23    252
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Мы видим, что в датасете есть 100 строк, где возраст клиентов равен 0, что является невозможным, но так как эти данные не будут в дальнейшем использоваться в исследовании - оставим их как есть.

In [15]:
df['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

Есть одна строка, где не указан пол клиента, но мы не будем использовать эти данные при дальнейшем исследовании, поэтому оставим столбец без изменений.

# Шаг 2.3. Изменение типов данных.

Данные в столбце `days_employed` имеют тип float64, но количество дней корректнее представлять в целочисленном виде, изменим тип данных на int.

In [16]:
import warnings
warnings.filterwarnings('ignore')

df['days_employed'] = df['days_employed'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  int64  
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


# Шаг 2.4. Удаление дубликатов.

In [17]:
# Найдём явные дубликаты
df.duplicated().sum()

54

In [18]:
# Удалим явные дубликаты
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [19]:
# Найдем неявные дубликаты
df['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [20]:
# Так как все дубликаты отличаются только регистром, приведем данные к единому виду с помощью метода str.lower().
df['education'] = df['education'].str.lower()
df['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [21]:
#Проверим другие столбцы датасета
df['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [22]:
df['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [23]:
df['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

По сути все цели получения кредита можно разделить на 4 большие группы (операции с автомобилем, операции с недвижимостью, проведение свадьбы, получение образования), но в данном случае мы не будем рассматривать цели как дубликаты.

In [24]:
# После обработки неявных дубликатов ещё раз проверим явные дубликаты 
df.duplicated().sum()

17

In [25]:
# Удалим явные дубликаты
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод:**
Дубликаты могут возникнуть в датасете по разным причинам, основными из них являются:
 - человеческий фактор
 - некорректное объединение данных из разных источников
 - некорректная предобработка данных
 - выгрузка одинаковых данных
 
 Выбор метода обработки дубликатов зависит от их категории: 
 - если дубликаты входят в группу явных дубликатов (т.е. строчки полностью повторятся) - используем метод `drop_duplicates()`, который полностью удаляет задублированные строки.
 - если дубликаты относятся к группе неявных - используем метод `unique()` для их нахождения и метод `str.lower()` для замены.
 

# Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [26]:
# Создадим 2 новых дотафрейма
education = df[['education','education_id']]
education = education.drop_duplicates().reset_index(drop = True) # удаляем дубликаты => получаем список уникальных значений
education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [27]:
family_status = df[['family_status','family_status_id']]
family_status = family_status.drop_duplicates().reset_index(drop = True) # удаляем дубликаты => получаем список уникальных значений
family_status

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [28]:
# Удалим столбцы education и family_status
df = df.drop(['education','family_status'],axis = 1)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,0,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,0,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,1,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,0,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


# Шаг 2.6. Категоризация дохода.

In [29]:
# Объявим функцию, которая распределит доход кредитополучателей по категориям
def income_category (income):
    if income <= 30000:
        return 'E'
    if income >= 30001 and income <= 50000:
        return 'D'
    if income >= 50000 and income <= 200000:
        return 'C'
    if income >= 200001 and income <= 1000000:
        return 'B'
    return 'A'

# Добавим столбец с категорией дохода в датафрейм
df['total_income_category'] = df['total_income'].apply(income_category)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875.639453,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885.952297,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763.565419,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525.971920,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823.934197,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856.832424,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,C


# Шаг 2.7. Категоризация целей кредита.

Ранее мы выяснили, что цели кредита можно разделить на 4 большие группы, с помощью функции распределим данные и создадим новый столбец - `purpose_category`

In [30]:
def purpose_group (purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образован' in purpose:
        return 'получение образования'
    if 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'

df['purpose_category'] = df['purpose'].apply(purpose_group)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875.639453,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885.952297,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763.565419,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525.971920,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823.934197,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856.832424,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,C,операции с недвижимостью


# Ответы на вопросы.

# Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [31]:
children_pivot = df.pivot_table(index = ['children'], values= 'debt', aggfunc = ['count','sum','mean'])
children_pivot['mean'] = children_pivot['mean']*100
children_pivot

,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,7.543822
1,4808,444,9.234609
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


Согласно данным клиенты с детьми возвращают кредит в срок премерно на 2% больше, чем клиенты без детей, при этом количество детей (1,2,3 или более) особо не влияет на % возврата кредита, среднее значение составляет 9,5%.

Важно отметить, что клиенты, у которых в семье 5 детей, ни разу не возвращали кредит в срок, но таких всего лишь 9 чел., что является очень малой выборкой, поэтому мы исключили эти данные из анализа.

# Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
import warnings
warnings.filterwarnings('ignore')

family_pivot = df.pivot_table(index = ['family_status_id'],values= 'debt', aggfunc = ['count','sum','mean'])
family_pivot['mean'] = family_pivot['mean']*100
family_status_pivot = family_status.merge(family_pivot, on = 'family_status_id', how = 'right')
family_status_pivot

,family_status,family_status_id,"(count, debt)","(sum, debt)","(mean, debt)"
0,женат / замужем,0,12261,927,7.560558
1,гражданский брак,1,4134,385,9.313014
2,вдовец / вдова,2,951,63,6.624606
3,в разводе,3,1189,84,7.064760
4,Не женат / не замужем,4,2796,273,9.763948


Диапазон возврата кредита составляет 7-9%, при этом % возврата больше у не женатых / не замужних клиентов, а также у клиентов, состоящих в гражданском браке. Наименьший % возврата у клиентов из группы "вдова/вдовец".

# Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
income_pivot = df.pivot_table(index = ['total_income_category'],values= 'debt', aggfunc = ['count','sum','mean'])
income_pivot['mean'] = income_pivot['mean']*100
income_pivot

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,8.000000
B,5014,354,7.060231
C,15921,1353,8.498210
D,349,21,6.017192
E,22,2,9.090909


Самый большой % возврата кредита у клиентов с уровнем дохода из категорий "А","С" и "Е", при этом, важно отметить, что количество клиентов из категории "А" и "Е" незначительно и для более точных выводов необходимо увеличить выборку по ним. Клиенты с уровнем дохода из категорий "В" и "D" возвращают кредит реже примерно на 1,5-2 %. 

# Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [34]:
purpose_pivot = df.pivot_table(index = ['purpose_category'],values= 'debt', aggfunc = ['count','sum','mean'])
purpose_pivot['mean'] = purpose_pivot['mean']*100
purpose_pivot

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,9.347978
операции с недвижимостью,10751,780,7.255139
получение образования,3988,369,9.252758
проведение свадьбы,2313,183,7.911803


Наименьший % возврата кредита по операциям, связанным с недвижимостью (7,25%) и проведением свадьбы (7,91%), наибольший - по операциям с автомобилем (9,36%) и получением образования (9,25%)

# Вопрос 5: Влияет ли уровень образования на возврат кредита в срок?

In [35]:
import warnings
warnings.filterwarnings('ignore')

education_pivot = df.pivot_table(index = ['education_id'],values= 'debt', aggfunc = ['count','sum','mean'])
education_pivot['mean'] = education_pivot['mean']*100
education_level_pivot = education.merge(education_pivot, on = 'education_id', how = 'right')
education_level_pivot

,education,education_id,"(count, debt)","(sum, debt)","(mean, debt)"
0,высшее,0,5227,278,5.318538
1,среднее,1,15075,1355,8.988391
2,неоконченное высшее,2,741,68,9.176788
3,начальное,3,282,31,10.992908
4,ученая степень,4,6,0,0.000000


Согласно данным люди с высшим образованием возвращают кредит в срок практически в 2 раза реже, чем люди со средним образованием или неоконченным высшем.

Важно отметить, что данных по клиентам с ученой степенью очень мало, поэтому они не принимались в расчёт при анализе.

# Общий вывод:

Целью нашего анализа была проверка определенных критериев на то, как они влияют на возврат кредита в срок.
    
Для того, чтобы верно их оценить, в первую очередь мы поработали с данными в целом, а именно: 
 - избавились от дубликатов, 
 - скорректировали типы данных,
 - выявили аномальные значения.

Далее для удобства анализа мы добавили дополнительные столбцы, в которых разделили уровень дохода и цели кредита на определенные категории.

В ходе исследования мы проверили несколько гипотез и установили, что:
    
1. Клиенты, которые состоят в гражданском браке или не состоят в браке вовсе возвращают кредит в срок примерно на 2% чаще, чем замужние/женатые клиенты или же клиенты в разводе. Реже всего возвращают кредит в срок клиенты из группы "вдова/вдовец". Таким образом, первая гипотеза о том, что люди состоящие в браке чаще возвращают кредит в срок не подтвердилась.

2. Клиенты с детьми возвращают кредит в срок чаще, чем клиенты без детей, при этом количество детей значительно не влияет на % возврата. Таким образом, вторая гипотеза полностью подтвердилась.

3. Клиенты со средним уровнем дохода (50 - 200 тыс.) возвращают кредит в срок на 1,5-2% чаще, чем клиенты с уровнем дохода из соседних диапазонов ("В" и "D"). Третья гипотеза не подтвердилась.


Дополнительно мы проверили, как влияют разные цели получения кредита и уровень образования клиентов на возврат кредита в срок: 
 - так если целью являются операции с недвижимостью или проведение свадьбы, % возврата относительно других групп снижается примерно на 1-2%,
 - а клиенты с высшим образованием практически в 2 раза реже возвращают кредит в срок, чем клиенты со среднем и неоконченным высшем образованием.